In [1]:
import numpy as np
import skimage.measure
# import pydicom
import vtk
from vtk.util import numpy_support
import os
import matplotlib.pyplot as plt
import cv2

In [ ]:
import gc
gc.collect()

In [9]:
plt.gray()
os.chdir('C:/Users/span/Documents/CNN/')
number1 = 32

<Figure size 432x288 with 0 Axes>

In [10]:
PathDicom = "01_perprocess/"+str(number1)+'/'+str(number1)+"_dl_input/"
reader = vtk.vtkDICOMImageReader()
reader.SetDirectoryName(PathDicom)
reader.Update()

# Load dimensions using `GetDataExtent`
_extent = reader.GetDataExtent()
ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

# Load spacing values
ConstPixelSpacing = reader.GetPixelSpacing()

# Get the 'vtkImageData' object from the reader
imageData = reader.GetOutput()
# Get the 'vtkPointData' object from the 'vtkImageData' object
pointData = imageData.GetPointData()
# Ensure that only one array exists within the 'vtkPointData' object
assert (pointData.GetNumberOfArrays()==1)
# Get the `vtkArray` (or whatever derived type) which is needed for the `numpy_support.vtk_to_numpy` function
arrayData = pointData.GetArray(0)

# Convert the `vtkArray` to a NumPy array
ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
# Reshape the NumPy array to 3D using 'ConstPixelDims' as a 'shape'
ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')

a_in = ArrayDicom.copy()

In [11]:
PathDicom = "01_perprocess/"+str(number1)+'/'+str(number1)+"_dl_mask/"
reader = vtk.vtkDICOMImageReader()
reader.SetDirectoryName(PathDicom)
reader.Update()

# Load dimensions using `GetDataExtent`
_extent = reader.GetDataExtent()
ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

# Load spacing values
ConstPixelSpacing = reader.GetPixelSpacing()

# Get the 'vtkImageData' object from the reader
imageData = reader.GetOutput()
# Get the 'vtkPointData' object from the 'vtkImageData' object
pointData = imageData.GetPointData()
# Ensure that only one array exists within the 'vtkPointData' object
assert (pointData.GetNumberOfArrays()==1)
# Get the `vtkArray` (or whatever derived type) which is needed for the `numpy_support.vtk_to_numpy` function
arrayData = pointData.GetArray(0)

# Convert the `vtkArray` to a NumPy array
ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
# Reshape the NumPy array to 3D using 'ConstPixelDims' as a 'shape'
ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')

a_mask = ArrayDicom.copy()

In [12]:
assert a_in.shape == a_mask.shape

In [13]:
import random
random.seed(a=1024, version=2)
list1 = list(range(a_in.shape[2]))
trainlist1 = random.sample(list1, k=round(a_in.shape[2]*0.6))
testlist1 = [x for x in list1 if (x not in trainlist1)]

In [14]:
from PIL import Image
from skimage.transform import rescale
# output  23_train_image
path1 = "02_data/train_image_re255/image/"
path3 = "02_data/val_image_re255/image/"

resize1 = 256

# output  23_train_image
for i in trainlist1:
    image = a_in[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
    img0 = Image.fromarray(image)
    img0.save(path1+str(number1)+"_taimage_"+str("{0:03}".format(i))+".png")
    
# output  23_test_image

for i in testlist1:
    image = a_in[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
    img0 = Image.fromarray(image)
    img0.save(path3+str(number1)+"_tsimage_"+str("{0:03}".format(i))+".png")

C:\Users\span\anaconda3\envs\tf-n-gpu\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
C:\Users\span\anaconda3\envs\tf-n-gpu\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


In [15]:
from PIL import Image
from skimage.transform import rescale
# re255 rescale to 0 - 255

path2 = "02_data/train_mask_re255/seg/"
path4 = "02_data/val_mask_re255/seg/"

# output  23_train_mask

for i in trainlist1:
    image = a_mask[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
    img0 = Image.fromarray(image)
    img0.save(path2+str(number1)+"_taimage_"+str("{0:03}".format(i))+".png")


# output  23_train_mask

for i in testlist1:
    image = a_mask[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
    img0 = Image.fromarray(image)
    img0.save(path4+str(number1)+"_tsimage_"+str("{0:03}".format(i))+".png")

C:\Users\span\anaconda3\envs\tf-n-gpu\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
C:\Users\span\anaconda3\envs\tf-n-gpu\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


In [15]:
# rescale to only 0 and 255

path2 = "02_data/train_mask_0255/seg/"
path4 = "02_data/val_mask_0255/seg/"
# output  23_train_mask

for i in trainlist1:
    image = a_mask[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image[image != 0] = 255
    img0 = Image.fromarray(image.astype(np.uint8))
    img0.save(path2+str(number1)+"_taimage_"+str("{0:03}".format(i))+".png")


# output  23_train_mask

for i in testlist1:
    image = a_mask[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image[image != 0] = 255
    img0 = Image.fromarray(image.astype(np.uint8))
    img0.save(path4+str(number1)+"_tsimage_"+str("{0:03}".format(i))+".png")

In [ ]:
import PIL.ImageOps
from PIL import Image
from skimage.transform import rescale
# output  23_train_image

path2 = "02_data/train_mask/23_train_mask/bg/"
path4 = "02_data/val_mask/23_test_mask/bg/"


# output  23_train_mask bg
for i in trainlist1:
    image = a_mask[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
    img0 = Image.fromarray(image)
    img0 = PIL.ImageOps.invert(img0)
    img0.save(path2+str(number1)+"_taimage_"+str("{0:03}".format(i))+".png")

# output  23_train_mask bg

for i in testlist1:
    image = a_mask[:, :, i].astype(np.uint8).copy()
    image = rescale(image, (resize1/a_in[:, :, i].shape[1]), anti_aliasing=False)
    image = (((image - image.min()) / (image.max() - image.min())) * 255.9).astype(np.uint8)
    img0 = Image.fromarray(image)
    img0 = PIL.ImageOps.invert(img0)
    img0.save(path4+str(number1)+"_tsimage_"+str("{0:03}".format(i))+".png")

In [ ]:
i1 = 25
i2 = 45
fig, (ax0, ax1, ax2, ax3) = plt.subplots(1, 4, figsize=(16, 6.4),
                                    sharex=True, sharey=True)
ax0.imshow(a_in[:, :, trainlist1[i1]], cmap='gray')
ax0.axis('off')
ax0.set_title('a in 1')
ax1.imshow(a_mask[:, :, trainlist1[i1]], cmap='gray')
ax1.axis('off')
ax1.set_title('a out 1')
ax2.imshow(a_in[:, :, trainlist1[i2]], cmap='gray') # 'magma'
ax2.axis('off')
ax2.set_title('a in 2')
ax3.imshow(a_mask[:, :, trainlist1[i2]], cmap='gray')
ax3.axis('off')
ax3.set_title('a out 2')

fig.tight_layout()
plt.show()

GENERATORS

CNN layer calculate 

In [ ]:
# [filter size, stride, padding]
#Assume the two dimensions are the same
#Each kernel requires the following parameters:
# - k_i: kernel size
# - s_i: stride
# - p_i: padding (if padding is uneven, right padding will higher than left padding; "SAME" option in tensorflow)
# 
#Each layer i requires the following parameters to be fully represented: 
# - n_i: number of feature (data layer has n_1 = imagesize )
# - j_i: distance (projected to image pixel distance) between center of two adjacent features
# - r_i: receptive field of a feature in layer i
# - start_i: position of the first feature's receptive field in layer i (idx start from 0, negative means the center fall into padding)

import math
convnet =   [[11,4,0],[3,2,0],[5,1,2],[3,2,0],[3,1,1],[3,1,1],[3,1,1],[3,2,0],[6,1,0], [1, 1, 0]]
layer_names = ['conv1','pool1','conv2','pool2','conv3','conv4','conv5','pool5','fc6-conv', 'fc7-conv']
imsize = 227

def outFromIn(conv, layerIn):
  n_in = layerIn[0]
  j_in = layerIn[1]
  r_in = layerIn[2]
  start_in = layerIn[3]
  k = conv[0]
  s = conv[1]
  p = conv[2]
  
  n_out = math.floor((n_in - k + 2*p)/s) + 1
  actualP = (n_out-1)*s - n_in + k 
  pR = math.ceil(actualP/2)
  pL = math.floor(actualP/2)
  
  j_out = j_in * s
  r_out = r_in + (k - 1)*j_in
  start_out = start_in + ((k-1)/2 - pL)*j_in
  return n_out, j_out, r_out, start_out
  
def printLayer(layer, layer_name):
  print(layer_name + ":")
  print("\t n features: %s \n \t jump: %s \n \t receptive size: %s \t start: %s " % (layer[0], layer[1], layer[2], layer[3]))
 
layerInfos = []
if __name__ == '__main__':
#first layer is the data layer (image) with n_0 = image size; j_0 = 1; r_0 = 1; and start_0 = 0.5
  print ("-------Net summary------")
  currentLayer = [imsize, 1, 1, 0.5]
  printLayer(currentLayer, "input image")
  for i in range(len(convnet)):
    currentLayer = outFromIn(convnet[i], currentLayer)
    layerInfos.append(currentLayer)
    printLayer(currentLayer, layer_names[i])
  print ("------------------------")
  layer_name = raw_input ("Layer name where the feature in: ")
  layer_idx = layer_names.index(layer_name)
  idx_x = int(raw_input ("index of the feature in x dimension (from 0)"))
  idx_y = int(raw_input ("index of the feature in y dimension (from 0)"))
  
  n = layerInfos[layer_idx][0]
  j = layerInfos[layer_idx][1]
  r = layerInfos[layer_idx][2]
  start = layerInfos[layer_idx][3]
  assert(idx_x < n)
  assert(idx_y < n)
  
  print ("receptive field: (%s, %s)" % (r, r))
  print ("center: (%s, %s)" % (start+idx_x*j, start+idx_y*j))